<h1>FCTM</h1>

<p><img alt="" src="https://www.nyu.edu/projects/yzhang/AlphaSpace/images/slide03.png" style="float:right; height:507px; width:1000px" /></p>

<p>This tutorial will introduce you to FCTM fragment centric topographical mapping of a single snapshot. For older version, click <a href="https://www.nyu.edu/projects/yzhang/AlphaSpace/" target="_blank">here</a>.</p>

<p>If you are familiar with AlphaSpace 1.0, here you will see some reference to its tutorial, otherwise you can ignore them.</p>


# 1. Program requirements

Python, SciPy, NumPy, Mdtraj, Jupyter

These dependencies should be installed automatically when you install AlphaSpace.

To view how to install, please proceed to [README.md](../README.md)

Now AlphaSpace 2.0 no longer requires Naccess for ASA calculation, it is replaced by [Shrake and Rupley](http://mdtraj.org/1.7.2/examples/solvent-accessible-surface-area.html).

# 2. Load PDB File

AlphaSpace 2.0 can process multiple snapshots from a simulation, but for now let's start with a single snapshot. We will use the mdm2-p53 complex, same as 1.0 tutorial.

AlphaSpace can map a complete protein surface or can be restricted to map just the interface between a receptor protein and a binding partner. The "receptor"/"protein" part and the "binder"/"ligand" part can be automatically identified, or manually set.

First Import the modules, you should see them

In [7]:
import alphaspace as al
import mdtraj
import os

In [8]:
# Load in receptor and ligand seperately using mdtraj. So you can load anything that is supported by mdtraj.
receptor = mdtraj.load("mdm2_p53/mdm2.pdb")
binder = mdtraj.load("mdm2_p53/p53.pdb")

In [9]:
# If you have the pdbqt file of the receptor, you can calculate the Vina Score. You can load the pdbqt file like this
al.annotateVinaAtomTypes(pdbqt="mdm2_p53/mdm2.pdbqt", receptor=receptor)

In [14]:
# Initialize a snapshot object, this will contain the receptor and the binder informations
ss = al.Snapshot()
ss.beta_cluster_dist = 1.3
ss.contact_cutoff = 1.6
# Run the snapshot object by feeding it receptor and binder mdtraj objects.
ss.run(receptor=receptor, binder=binder)

Vina Atom Type found, calculating BScore


In [17]:
# You can use a simple for loop to iterate over all pockets found in the snapshot

for p in sorted(ss.pockets, key=lambda i: i.space, reverse=True):
    if p.isContact:
        print(
            "{} has space of {:.1f}, and VinaScore of {:.1f} and {:.1f}% occupied".format(p, (p.space), p.score,
                                                                                          p.occupancy * 100)
        )


Pocket 61 has space of 223.6, and VinaScore of -5.6 and 100.0% occupied
Pocket 58 has space of 159.9, and VinaScore of -4.4 and 56.5% occupied
Pocket 59 has space of 147.0, and VinaScore of -2.8 and 96.2% occupied
Pocket 57 has space of 132.6, and VinaScore of -3.7 and 9.9% occupied
Pocket 21 has space of 111.8, and VinaScore of -3.7 and 60.0% occupied
Pocket 55 has space of 67.6, and VinaScore of -2.3 and 90.1% occupied
